In [57]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
#from pyxdameraulevenshtein import damerau_levenshtein_distance
from fastDamerauLevenshtein import damerauLevenshtein

pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv('../data_archive/all_task_environments_new.csv', header=0, skiprows=[1])

In [4]:
objects = ['t', 'n', 's', 'p', 'c']

coordinates = {'c': (1,4,4),
              'n': (1,1,2),
              'p': (1,4,4),
              's': (1,2,2),
              't': (1,1,2),
              'start': (2,1,2),
              'table': (4,3,2)}

c1 = {'c': 1.5,
    'n': 1.0,
    'p': 1.5,
    's': 1.5,
    't': 1.0}

k1 = {'c': 1.0,
    'n': 0.4,
    'p': 0.4,
    's': 1.0,
    't': 0.3}

start_coordinates = [[2,1.5,2],[4,3,2],[4,3,2],[4,3,2],[4,3,2]]

In [5]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,], steps=1):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        #print(minval)
        del possible_items[minval]
        
        if steps == 2 and bool(possible_items) == True:
            for obj in possible_items.keys():
                possible_items[obj] = ((distance.euclidean(
                                    new_coords[minval],
                                    new_coords[obj])
                                    )
                                    ** k[obj]) * c[obj]
            #print(possible_items)
            minval_step2 = min(possible_items.values())
            minval_step2 = [k for k, v in possible_items.items() if v == minval_step2]
            minval_step2 = random.choice(minval_step2) # choose prediction randomly if multiple items have same cost
            #print(minval_step2 + '____')
            del possible_items[minval_step2]
            prediction.append(''.join([minval, minval_step2]))
            coord_index += 1
            
        else:
            prediction.append(minval)
          
        coord_index += 1
        
    return prediction

In [6]:
predict_sequence(objects, coordinates, start_coordinates, c1, k1, steps=2)

['tn', 'pc', 's']

In [24]:
def get_average(objects, coordinates, start_coordinates, c, k, dimension, sequence, steps=1):

    edit_list = []

    for x in range(0,100):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerauLevenshtein(sequence,result,similarity=False)
        edit_list.append(dl)
    
    avg = np.mean(edit_list)
    median = np.median(edit_list)
    return avg, median

In [35]:
def get_avg_editdist(data, steps=1):
    results = pd.DataFrame()
    #dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]
    dimensions = [[2,'xy']]
    
    for row in range(0,len(df)):
        objects = list(df.at[row,'objects'].split(','))
        strong_k = list(df.at[row,'strong_k'].split(','))
        mid_k = list(df.at[row,'mid_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
        start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
        sequence = str(df.at[row,'sequence'])
        
        #for k in np.arange(0.0,1.0,0.1):
        for k in np.arange(0.1,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else 1.0 for obj in objects}
                    
            for c in np.arange(1.1,2.0,0.1):
                c = round(c, 1)
                c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
                
                for dim in dimensions:                
                    # get average edit distance
                    edit_dist, median = get_average(objects, coordinates, start_coordinates, c1, k1, dim, sequence)
                    edit_dist = edit_dist / len(sequence)
                    median = median / len(sequence)
                    
                    #params = 'c: ' + str(c) + ', k: ' + str(k_strong) + ',' + str(k_mid) + ', dim: ' + str(dim[1])
                    params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                    results.at[row,params] = median
                    
    return results

### Generate predictions for 1 and 2 steps of planning

In [39]:
#%%timeit -n1 -r1
# generate data for 2 steps of planning
results_new = get_avg_editdist(df, steps=2)

In [40]:
results_new

,"c: 1.1; k: 0.1,0.2; xy","c: 1.2; k: 0.1,0.2; xy","c: 1.3; k: 0.1,0.2; xy","c: 1.4; k: 0.1,0.2; xy","c: 1.5; k: 0.1,0.2; xy","c: 1.6; k: 0.1,0.2; xy","c: 1.7; k: 0.1,0.2; xy","c: 1.8; k: 0.1,0.2; xy","c: 1.9; k: 0.1,0.2; xy","c: 1.1; k: 0.2,0.3; xy",...,"c: 1.9; k: 0.7,0.8; xy","c: 1.1; k: 0.8,0.9; xy","c: 1.2; k: 0.8,0.9; xy","c: 1.3; k: 0.8,0.9; xy","c: 1.4; k: 0.8,0.9; xy","c: 1.5; k: 0.8,0.9; xy","c: 1.6; k: 0.8,0.9; xy","c: 1.7; k: 0.8,0.9; xy","c: 1.8; k: 0.8,0.9; xy","c: 1.9; k: 0.8,0.9; xy"
0,0.200000,0.000000,0.200000,0.100000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,...,0.000000,0.400000,0.200000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000
1,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000,0.400000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
3,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.333333,0.333333,0.333333,0.666667,0.666667,0.500000,0.333333,0.666667,0.500000,0.333333
5,0.600000,0.600000,0.500000,0.600000,0.600000,0.400000,0.600000,0.400000,0.400000,0.400000,...,0.500000,0.400000,0.600000,0.600000,0.400000,0.400000,0.500000,0.600000,0.600000,0.400000
6,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
7,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333


In [41]:
# generate data for 1 step of planning
results1 = get_avg_editdist(df, steps=1)

In [42]:
def get_lowest_error(results):
    for col in list(results):
        results.loc['mean',col] = results[col].mean()
        results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    lowest_median = min(results.loc['median'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

### Compare results

In [47]:
# results for 2 steps
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx, results_mean['c: 1.4; k: 0.2,0.3; xy']['median']

(0.3774470899470899,
 Index(['c: 1.4; k: 0.2,0.3; xy'], dtype='object'),
 0.3774470899470899)

In [49]:
# results for 1 step
lowest1, lowest_idx1, list_mean1, results_mean1 = get_lowest_error(results1)
lowest1, lowest_idx1, results_mean1['c: 1.4; k: 0.2,0.3; xy']['median']

(0.3691137566137566,
 Index(['c: 1.4; k: 0.2,0.3; xy'], dtype='object'),
 0.36911375661375656)

In [51]:
# results for 2 steps
list_xy = []
median_xy = []

for col in results_mean:
    if col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
        median_xy.append(results_mean.at['median',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(median_xy)
print('Average xy: ' + str(avg_xy) + ', median: ' + str(med_xy))

Average xy: 0.4191484227680603, median: 0.40626653439153443


In [52]:
# results for 1 step
list_xy1 = []
median_xy1 = []

for col in results_mean1:
    if col.split(';')[2].strip() == 'xy':
        list_xy1.append(results_mean1.at['mean',col])
        median_xy1.append(results_mean1.at['median',col])
        
avg_xy1 = np.mean(list_xy1)
med_xy1 = np.median(median_xy1)
print('Average xy: ' + str(avg_xy1) + ', median: ' + str(med_xy1))

Average xy: 0.41772089650862354, median: 0.4


In [53]:
avg_1step = 0
avg_2steps = 0
tie = 0

for i in range(0,len(list_mean)):
    if list_mean[i] > list_mean1[i]:
        avg_1step += 1
    elif list_mean[i] < list_mean1[i]:
        avg_2steps +=1
    elif list_mean[i] == list_mean1[i]:
        tie += 1

print(avg_1step, avg_2steps, tie)

40 31 1


### Stat analysis

In [71]:
# compare 1 vs 2 steps: mean + median

stat, p = wilcoxon(list_xy, list_xy1, zero_method='wilcox')
print('Wilcoxon for xy: T = %.3f, p = %.3f' % (stat, p))

stat, p = wilcoxon(median_xy, median_xy1, zero_method='wilcox')
print('Wilcoxon for xy median: T = %.3f, p = %.3f' % (stat, p))

Wilcoxon for xy: T = 1121.000, p = 0.368
Wilcoxon for xy median: T = 726.000, p = 0.230


### Plot 1 vs 2 steps (xy)

In [55]:
# get data for plot (results for both)
c2 = [float(x[3:6]) for x in results_new.columns.tolist()]
k2 = [float(x[11:14]) for x in results_new.columns.tolist()]
error2 = results_mean.loc['mean'].tolist()
mean2 = results_mean.loc['median'].tolist()

c1 = [float(x[3:6]) for x in results1.columns.tolist()]
k1 = [float(x[11:14]) for x in results1.columns.tolist()]
error1 = results_mean1.loc['mean'].tolist()
mean1 = results_mean1.loc['median'].tolist()

In [74]:
%matplotlib qt

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot
step1 = ax2.scatter(c1, k1, error1, alpha=0.5, s=38, c="#277f8e")
step2 = ax2.scatter(c2, k2, error2, alpha=0.5, s=38, c="#4ac16d")

# set labels
ax2.set_ylabel('parameter k', fontsize=12, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=12, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=12, labelpad=7)

# plot horizontal plane for baseline
#xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#zz = xx * 0 + 0.603
#ax2.plot_surface(xx, yy, zz, alpha=0.5)

ax2.legend(loc=(0.5,0.5,0.5), frameon=0)

# add legend
plt.legend((step1, step2),
           ('1 step', '2 steps'),
           scatterpoints=4,
           fontsize=12,
           loc='lower left', fancybox=True, framealpha=0.2)

plt.show()

/tmp/ipykernel_73561/1614647322.py:6: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
/tmp/ipykernel_73561/1614647322.py:7: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
/tmp/ipykernel_73561/1614647322.py:8: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [61]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]

### Plot data distribution for xy

In [70]:
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True)

sns.violinplot(error1,
               palette=custom_cmap2[1:],
               cut=1,
               saturation=0.8,
               inner='quartiles',
               ax=ax1)

ax1.set_xlabel('1 step', fontsize=24)
ax1.set_ylabel('normalized Damerau-Levenshtein distance', fontsize=24)

sns.violinplot(error2,
               palette=custom_cmap2[2:],
               cut=1,
               saturation=0.8,
               inner='quartiles',
               ax=ax2)

ax2.set_xlabel('2 steps', fontsize=24)
#ax2.set_ylabel('normalized Damerau-Levenshtein distance', fontsize=24)

plt.setp(ax1.collections, alpha=0.5)
plt.setp(ax2.collections, alpha=0.5)

plt.show()